In [1]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [2]:


# Define the mocap data (simplified for brevity, add the full data in your actual use case)
data = np.array([
    [0.3, 1, 0.7552130000, 4],
    [0.6, 2, 0.7525270000, 5],
    [0.0, 4, 0.7501210000, 7],
])

# Fix the last time step
dt = data[0][0]
prev_tim = data[-2][0]
data[-1][0] = prev_tim + dt

# Duplicate the data
duplicated_data = np.tile(data, (2, 1))

# Calculate the displacement to add to the root positions in the second half
# Only the time and the root position need to be adjusted
displacement = data[-1, :4]

# Adjust the root positions in the second half
duplicated_data[len(data):, :4] += displacement

print(duplicated_data)



[[ 0.3       1.        0.755213  4.      ]
 [ 0.6       2.        0.752527  5.      ]
 [ 0.9       4.        0.750121  7.      ]
 [ 1.2       5.        1.505334 11.      ]
 [ 1.5       6.        1.502648 12.      ]
 [ 1.8       8.        1.500242 14.      ]]


In [3]:
from utils.DataConverter import DataConverter

converterpunch = DataConverter("motions/humanoid3d_punch.txt",2)
converterwalk = DataConverter("motions/humanoid3d_walk.txt",5)




In [4]:
from utils.SimpleConverter import SimpleConverter
trajectory = SimpleConverter("motions/humanoid3d_punch_duplicated.txt")
trajectory.load_mocap()

In [5]:
trajectory.duration_dict

{0: [0.0, 0.0333333015],
 1: [0.03333330154418945, 0.0333333015],
 2: [0.0666666030883789, 0.0333333015],
 3: [0.09999990463256836, 0.0333333015],
 4: [0.1333332061767578, 0.0333333015],
 5: [0.16666650772094727, 0.0333333015],
 6: [0.19999980926513672, 0.0333333015],
 7: [0.23333311080932617, 0.0333333015],
 8: [0.2666664123535156, 0.0333333015],
 9: [0.2999997138977051, 0.0333333015],
 10: [0.33333301544189453, 0.0333333015],
 11: [0.366666316986084, 0.0333333015],
 12: [0.39999961853027344, 0.0333333015],
 13: [0.4333329200744629, 0.0333333015],
 14: [0.46666622161865234, 0.0333333015],
 15: [0.4999995231628418, 0.0333333015],
 16: [0.5333328247070312, 0.0333333015],
 17: [0.5666661262512207, 0.0333333015],
 18: [0.5999994277954102, 0.0333333015],
 19: [0.6333327293395996, 0.0333333015],
 20: [0.6666660308837891, 0.0333333015],
 21: [0.6999993324279785, 0.0333333015],
 22: [0.733332633972168, 0.0333333015],
 23: [0.7666659355163574, 0.0333333015],
 24: [0.7999992370605469, 0.0333333